<table align="center">
  <td align="center"><a target="_blank" href="http://introtodeeplearning.com">
        <img src="http://introtodeeplearning.com/images/colab/mit.png" style="padding-bottom:5px;" />
      Visit MIT Deep Learning</a></td>
  <td align="center"><a target="_blank" href="https://colab.research.google.com/github/aamini/introtodeeplearning/blob/master/lab3/solutions/RL_Solution.ipynb">
        <img src="http://introtodeeplearning.com/images/colab/colab.png?v2.0"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/aamini/introtodeeplearning/blob/master/lab3/solutions/RL_Solution.ipynb">
        <img src="http://introtodeeplearning.com/images/colab/github.png"  height="70px" style="padding-bottom:5px;"  />View Source on GitHub</a></td>
</table>

# Copyright Information

In [ ]:
# Copyright 2020 MIT 6.S191 Introduction to Deep Learning. All Rights Reserved.
# 
# Licensed under the MIT License. You may not use this file except in compliance
# with the License. Use and/or modification of this code outside of 6.S191 must
# reference:
#
# © MIT 6.S191: Introduction to Deep Learning
# http://introtodeeplearning.com
#

# Обучение с подкреплением

Обучение с подкреплением (RL) - это подмножество машинного обучения, которое ставит проблемы обучения как взаимодействие между агентами и средой. Часто предполагается, что агенты не имеют предварительных знаний о мире, поэтому они должны научиться ориентироваться в окружающей среде, оптимизируя функцию вознаграждения. В среде агент может предпринимать определенные действия и получать обратную связь в виде положительного или отрицательного вознаграждения за свое решение. Таким образом, цикл обратной связи агента напоминает идею "проб и ошибок", или то, как ребенок может научиться различать "хорошие" и "плохие" действия.

С практической точки зрения, наш агент RL будет взаимодействовать с окружающей средой, выполняя действие на каждом временном шаге, получая соответствующее вознаграждение и обновляя свое состояние в соответствии с тем, чему он "научился".  

![alt text](https://www.kdnuggets.com/images/reinforcement-learning-fig1-700.jpg)

Хотя конечной целью обучения с подкреплением является обучение агентов действиям в реальном, физическом мире, игры представляют собой удобный полигон для разработки алгоритмов и агентов RL. Игры обладают некоторыми свойствами, которые делают их особенно подходящими для RL: 

1.   Во многих случаях игры имеют идеально описываемое окружение. Например, все правила игры в шахматы могут быть формально записаны и запрограммированы в симуляторе шахматной игры;
2.   Игры поддаются массовому распараллеливанию. Поскольку они не требуют выполнения в реальном мире, одновременные среды можно запускать на больших кластерах данных; 
3.   Более простые сценарии в играх позволяют быстро создавать прототипы. Это ускоряет разработку алгоритмов, которые в конечном итоге могут работать в реальном мире; и
4. ... Игры - это весело! 

Обучение с подкреплением принципиально отличается тем, что мы обучаем алгоритм глубокого обучения для управления действиями нашего агента RL, который пытается в своем окружении найти оптимальный способ достижения цели. Цель обучения RL-агента - определить наилучший следующий шаг, который нужно сделать, чтобы получить наибольшую конечную прибыль или доход. В этой лабораторной работе мы сосредоточимся на создании алгоритма обучения с подкреплением для освоения двух различных сред с разной сложностью. 

**Тележка с маятником**:   Уравновесить шест, выступающий из тележки, в вертикальном положении, перемещая основание только влево или вправо.

Давайте начнем! Сначала мы импортируем TensorFlow, пакет курса и некоторые зависимости.


In [ ]:
!apt-get install -y xvfb python-opengl x11-utils > /dev/null 2>&1
!pip install gym pyvirtualdisplay scikit-video > /dev/null 2>&1

%tensorflow_version 2.x
import tensorflow as tf

import numpy as np
import base64, io, time, gym
import IPython, functools
import matplotlib.pyplot as plt
from tqdm import tqdm

!pip install mitdeeplearning
import mitdeeplearning as mdl

Прежде чем мы погрузимся в процесс, давайте сделаем шаг назад и опишем наш подход, который в целом применим к проблемам обучения с усилением в целом:

1. **Инициализируем среду и агента**: здесь мы опишем различные наблюдения и действия, которые агент может выполнять в среде.
2. **Определить память нашего агента**: это позволит агенту помнить свои прошлые действия, наблюдения и вознаграждения.
3. **Определите функцию вознаграждения**: описывает вознаграждение, связанное с действием или последовательностью действий.
4. **Определите алгоритм обучения**: он будет использоваться для подкрепления хорошего поведения агента и предотвращения плохого поведения.


# Тележка с маятником

## 1.1 Определение среды и агента Cartpole

### Окружение 

Для моделирования среды для задач Cartpole и Pong мы будем использовать инструментарий, разработанный OpenAI под названием [OpenAI Gym] (https://gym.openai.com/). Он предоставляет несколько предустановленных сред для обучения и тестирования агентов обучения с подкреплением, включая среды для классических задач управления физикой, видеоигр Atari и симуляторов роботов. Для доступа к среде Cartpole мы можем использовать `env = gym.make("CartPole-v0")`, доступ к которой мы получили при импорте пакета `gym`. Мы можем создавать различные [окружения](https://gym.openai.com/envs/#classic_control), передавая имя окружения в функцию `make`.

Одна из проблем, с которой мы можем столкнуться при разработке алгоритмов RL, заключается в том, что многие аспекты процесса обучения по своей природе случайны: инициализация состояний игры, изменения в окружающей среде и действия агента. Поэтому для обеспечения некоторого уровня воспроизводимости может быть полезно задать начальную "затравку" для среды. Подобно тому, как вы можете использовать `numpy.random.seed`, мы можем вызвать сравнимую функцию в gym, `seed`, с нашей определенной средой, чтобы гарантировать, что случайные переменные среды инициализируются одинаково каждый раз.

In [ ]:
### Instantiate the Cartpole environment ###

env = gym.make("CartPole-v0")
env.seed(1)

В игре Cartpole шест крепится неактивируемым шарниром к тележке, которая движется по дорожке без трения. Столб стоит вертикально, и задача состоит в том, чтобы не дать ему упасть. Система управляется путем приложения к тележке силы +1 или -1. За каждый временной интервал, в течение которого столб остается в вертикальном положении, выдается вознаграждение +1. Эпизод заканчивается, когда шест отклоняется от вертикали более чем на 15 градусов или тележка перемещается более чем на 2,4 единицы от центра дорожки. Визуальная схема окружения полюса тележки показана ниже:

<img width="400px" src="https://danielpiedrahita.files.wordpress.com/2017/02/cart-pole.png"></img>.

Учитывая эту установку для окружения и цель игры, мы можем подумать о том: 1) какие наблюдения помогают определить состояние среды; 2) какие действия может предпринять агент. 

Во-первых, давайте рассмотрим пространство наблюдений. В этой среде, нашими наблюдениями являются:

1. Положение тележки
2. Скорость тележки
3. Угол поворота шеста
4. Скорость вращения шеста

Мы можем подтвердить размер пространства, запросив область видимости текущего пространства:


In [ ]:
n_observations = env.observation_space
print("Environment has observation space =", n_observations)

Во-вторых, мы рассматриваем пространство действий. На каждом временном шаге агент может двигаться либо вправо, либо влево. И снова мы можем подтвердить размер пространства действий путем опроса окружающей среды:

In [ ]:
n_actions = env.action_space.n
print("Number of possible actions that the agent can choose from =", n_actions)

### Агент Cartpole

Теперь, когда мы инстанцировали среду и поняли размерность пространств наблюдений и действий, мы готовы определить нашего агента. В глубоком обучении с подкреплением агента определяет глубокая нейронная сеть. Эта сеть принимает на вход наблюдение среды и выдает вероятность выполнения каждого из возможных действий. Поскольку Cartpole определяется низкоразмерным пространством наблюдений, для нашего агента должна хорошо подойти простая нейронная сеть с обратной связью. Мы определим ее с помощью API `Sequential`.


In [ ]:
### Define the Cartpole agent ###

# Defines a feed-forward neural network
def create_cartpole_model():
  model = tf.keras.models.Sequential([
      # First Dense layer
      tf.keras.layers.Dense(units=32, activation='relu'),

      # TODO: Define the last Dense layer, which will provide the network's output.
      # Think about the space the agent needs to act in!
      tf.keras.layers.Dense(units=n_actions, activation=None) # TODO
      # [TODO Dense layer to output action probabilities]
  ])
  return model

cartpole_model = create_cartpole_model()

Теперь, когда мы определили основную архитектуру сети, мы определим *функцию действия*, которая выполняет прямой проход по сети, учитывая набор наблюдений, и делает выборку из выходных данных. Эта выборка из выходных вероятностей будет использоваться для выбора следующего действия для агента. 

**Критически важно, что эта функция действия является абсолютно общей - мы будем использовать эту функцию как для Cartpole, и она также применима к другим задачам RL!

In [ ]:
### Define the agent's action function ###

# Function that takes observations as input, executes a forward pass through model, 
#   and outputs a sampled action.
# Arguments:
#   model: the network that defines our agent
#   observation: observation which is fed as input to the model
# Returns:
#   action: choice of agent action
def choose_action(model, observation):
  # add batch dimension to the observation
  observation = np.expand_dims(observation, axis=0)

  '''TODO: feed the observations through the model to predict the log probabilities of each possible action.'''
  logits = model.predict(observation) # TODO
  # logits = model.predict('''TODO''')
  
  # pass the log probabilities through a softmax to compute true probabilities
  prob_weights = tf.nn.softmax(logits).numpy()
  
  '''TODO: randomly sample from the prob_weights to pick an action.
  Hint: carefully consider the dimensionality of the input probabilities (vector) and the output action (scalar)'''
  action = np.random.choice(n_actions, size=1, p=prob_weights.flatten())[0] # TODO
  # action = np.random.choice('''TODO''', size=1, p=''''TODO''')['''TODO''']

  return action

## 1.2 Определение памяти агента

Теперь, когда мы инстанцировали среду и определили архитектуру сети агента и функцию действия, мы готовы перейти к следующему шагу в нашем рабочем процессе RL:
1. **Инициализация среды и агента**: здесь мы опишем различные наблюдения и действия, которые агент может выполнять в среде.
2. **Определите память нашего агента**: это позволит агенту помнить свои прошлые действия, наблюдения и награды.
3. **Определите алгоритм обучения**: он будет использоваться для подкрепления хорошего поведения агента и предотвращения плохого поведения.

В обучении с подкреплением обучение происходит параллельно с действиями агента в окружающей среде; под *эпизодом* понимается последовательность действий, которая заканчивается некоторым конечным состоянием, например, падением столба или падением тележки. Агент должен запомнить все свои наблюдения и действия, чтобы после завершения эпизода научиться "подкреплять" хорошие действия и наказывать нежелательные действия посредством обучения. Наш первый шаг - определить простой буфер памяти, который содержит наблюдения, действия и полученные вознаграждения агента за данный эпизод. 

**Опять же, обратите внимание на модульность этого буфера памяти - он может и будет применяться и для других задач RL!**.

In [ ]:
### Agent Memory ###

class Memory:
  def __init__(self): 
      self.clear()

  # Resets/restarts the memory buffer
  def clear(self): 
      self.observations = []
      self.actions = []
      self.rewards = []

  # Add observations, actions, rewards to memory
  def add_to_memory(self, new_observation, new_action, new_reward): 
      self.observations.append(new_observation)
      '''TODO: update the list of actions with new action'''
      self.actions.append(new_action) # TODO
      # ['''TODO''']
      '''TODO: update the list of rewards with new reward'''
      self.rewards.append(new_reward) # TODO
      # ['''TODO''']
        
memory = Memory()

## 1.3 Функция вознаграждения

Мы почти готовы начать алгоритм обучения нашего агента! Следующим шагом будет вычисление вознаграждения нашего агента в процессе его действий в окружающей среде. Поскольку мы (и агент) не знаем, закончится ли игра или задание (т.е. когда упадет столб) и когда это произойдет, полезно подчеркнуть получение вознаграждения **сейчас**, а не позже в будущем - в этом заключается идея дисконтирования (**Дисконтирование — определение стоимости денежного потока путём приведения стоимости всех выплат к определённому моменту времени.**). Это понятие схоже с дисконтированием денег в случае с процентами. Как вы помните из лекции, мы используем дисконтирование вознаграждения, чтобы отдать большее предпочтение получению вознаграждения сейчас, а не позже в будущем. Идея дисконтирования вознаграждения похожа на дисконтирование денег в случае с процентами.

Чтобы вычислить ожидаемое кумулятивное вознаграждение, известное как **доходность**, на данном временном шаге в эпизоде обучения, мы суммируем дисконтированные вознаграждения, ожидаемые на данном временном шаге $t$, в рамках эпизода обучения и проецируя их в будущее. Мы определяем доходность (кумулятивное вознаграждение) на временном шаге $t$, $R_{t}$ как:

>$R_{t}=\sum_{k=0}^\infty\gamma^kr_{t+k}$

где $0 < \gamma < 1$ - коэффициент дисконтирования, $r_{t}$ - вознаграждение на временном шаге $t$, а индекс $k$ увеличивает проекцию в будущее в рамках одного эпизода обучения. Интуитивно можно представить, что эта функция обесценивает любые вознаграждения, полученные на более поздних временных шагах, что заставляет агента отдавать приоритет получению вознаграждения сейчас. Поскольку мы не можем расширять эпизоды до бесконечности, на практике вычисления будут ограничены количеством временных шагов в эпизоде - после этого вознаграждение принимается равным нулю.

Обратите внимание на форму этой суммы - нам придется проявить смекалку при реализации этой функции. В частности, нам нужно будет инициализировать массив нулей длиной в количество временных шагов и заполнять его реальными значениями дисконтированного вознаграждения по мере того, как мы будем перебирать вознаграждения из эпизода, которые будут сохранены в памяти агента. В конечном итоге нас интересует, какие действия лучше по сравнению с другими действиями, предпринятыми в этом эпизоде, поэтому мы нормализуем наши вычисленные вознаграждения, используя среднее значение и стандартное отклонение вознаграждений по всему учебному эпизоду.


In [ ]:
### Reward function ###

# Helper function that normalizes an np.array x
def normalize(x):
  x -= np.mean(x)
  x /= np.std(x)
  return x.astype(np.float32)

# Compute normalized, discounted, cumulative rewards (i.e., return)
# Arguments:
#   rewards: reward at timesteps in episode
#   gamma: discounting factor
# Returns:
#   normalized discounted reward
def discount_rewards(rewards, gamma=0.95): 
  discounted_rewards = np.zeros_like(rewards)
  R = 0
  for t in reversed(range(0, len(rewards))):
      # update the total discounted reward
      R = R * gamma + rewards[t]
      discounted_rewards[t] = R
      
  return normalize(discounted_rewards)

## 1.4 Алгоритм обучения

Теперь мы можем начать определять алгоритм обучения, который будет использоваться для подкрепления хорошего поведения агента и препятствования плохому поведению. В этой лабораторной работе мы сосредоточимся на методах *policy gradient*, которые направлены на **максимизацию** вероятности действий, приводящих к большим вознаграждениям. Эквивалентно это означает, что мы хотим **минимизировать** отрицательную вероятность этих же действий. Мы достигаем этого, просто **масштабируя** вероятности на соответствующее вознаграждение - эффективно усиливая вероятность действий, которые приводят к большим вознаграждениям.

Поскольку функция log монотонно возрастает, это означает, что минимизация **отрицательной вероятности** эквивалентна минимизации **отрицательной log-вероятности**.  Напомним, что мы можем легко вычислить отрицательное лог-вероятность дискретного действия, оценив его [softmax cross entropy](https://www.tensorflow.org/api_docs/python/tf/nn/sparse_softmax_cross_entropy_with_logits). Как и в контролируемом обучении, мы можем использовать методы стохастического градиентного спуска для достижения желаемой минимизации. 

Начнем с определения функции потерь.

In [ ]:
### Loss function ###

# Arguments:
#   logits: network's predictions for actions to take
#   actions: the actions the agent took in an episode
#   rewards: the rewards the agent received in an episode
# Returns:
#   loss
def compute_loss(logits, actions, rewards): 
  '''TODO: complete the function call to compute the negative log probabilities'''
  neg_logprob = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=actions) # TODO
  # neg_logprob = tf.nn.sparse_softmax_cross_entropy_with_logits(logits='''TODO''', labels='''TODO''')
  
  '''TODO: scale the negative log probability by the rewards'''
  loss = tf.reduce_mean( neg_logprob * rewards ) # TODO
  # loss = tf.reduce_mean('''TODO''')
  return loss

Теперь давайте используем функцию потерь для определения шага обучения нашего алгоритма обучения:

In [ ]:
### Training step (forward and backpropagation) ###

def train_step(model, optimizer, observations, actions, discounted_rewards):
  with tf.GradientTape() as tape:
      # Forward propagate through the agent network
      logits = model(observations)

      '''TODO: call the compute_loss function to compute the loss'''
      loss = compute_loss(logits, actions, discounted_rewards) # TODO
      # loss = compute_loss('''TODO''', '''TODO''', '''TODO''')

  '''TODO: run backpropagation to minimize the loss using the tape.gradient method'''
  grads = tape.gradient(loss, model.trainable_variables) # TODO
  # grads = tape.gradient('''TODO''', model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))


## 1.5 Запустите Cartpole!

Не имея никаких предварительных знаний об окружающей среде, агент начнет учиться балансировать шест на тележке, основываясь только на обратной связи, получаемой из окружающей среды! Определив, как наш агент может двигаться, как он принимает новые наблюдения и как он обновляет свое состояние, мы увидим, как он постепенно учится политике действий для оптимизации балансировки шеста как можно дольше. Для этого мы проследим, как вознаграждение изменяется в зависимости от обучения - как должно меняться вознаграждение по мере обучения?

In [ ]:
### Cartpole training! ###

# Learning rate and optimizer
learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate)

# instantiate cartpole agent
cartpole_model = create_cartpole_model()

# to track our progress
smoothed_reward = mdl.util.LossHistory(smoothing_factor=0.9)
plotter = mdl.util.PeriodicPlotter(sec=2, xlabel='Iterations', ylabel='Rewards')

if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists
for i_episode in range(200):

  plotter.plot(smoothed_reward.get())

  # Restart the environment
  observation = env.reset()
  memory.clear()

  while True:
      # using our observation, choose an action and take it in the environment
      action = choose_action(cartpole_model, observation)
      next_observation, reward, done, info = env.step(action)
      # add to memory
      memory.add_to_memory(observation, action, reward)
      
      # is the episode over? did you crash or do so well that you're done?
      if done:
          # determine total reward and keep a record of this
          total_reward = sum(memory.rewards)
          smoothed_reward.append(total_reward)
          
          # initiate training - remember we don't know anything about how the 
          #   agent is doing until it has crashed!
          train_step(cartpole_model, optimizer, 
                     observations=np.vstack(memory.observations),
                     actions=np.array(memory.actions),
                     discounted_rewards = discount_rewards(memory.rewards))
          
          # reset the memory
          memory.clear()
          break
      # update our observatons
      observation = next_observation

Чтобы получить представление о работе нашего агента, мы можем сохранить видео, на котором обученная модель работает над балансировкой шеста. Поймите, что это совершенно новая среда, которую агент раньше не видел!

Давайте покажем сохраненное видео, чтобы посмотреть, как наш агент справился с задачей!


In [ ]:
saved_cartpole = mdl.lab3.save_video_of_model(cartpole_model, "CartPole-v0")
mdl.lab3.play_video(saved_cartpole)

Как работает агент? Могли бы вы тренировать его в течение более короткого времени и при этом показывать хорошие результаты? Думаете ли вы, что более длительная тренировка поможет еще больше? 